# Feature Extraction

We use this module to extract the features we used to feed into NN. Remember, take all the data into consideration.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# Importing the data
col_index = [1,2,3,4,6,7,9,10,12,13,15,16]
col_names = ['re_time','re_ID','re_x','re_y','tr_time','tr_ID','tr_x','tr_y','tr_vx','tr_vy','RSSI','Label']
data_atk1 = pd.read_csv('dataset/attack1withlabels.csv', usecols = col_index, header = None, names = col_names)
data_atk1 = data_atk1.dropna(axis = 0, how = 'any')

In [16]:
# Extract the sender ID and receiver ID and get their information. 
recevier_ID = np.unique(np.array(data_atk1.iloc[:,1]))
sender_ID = np.unique(np.array(data_atk1.iloc[:,5]))
recevier_ID

array([  10,   16,   22,   28,   34,   40,   46,   52,   58,   64,   70,
         76,   82,   88,   94,  100,  106,  118,  124,  130,  136,  142,
        148,  154,  160,  166,  172,  178,  184,  190,  196,  202,  208,
        214,  220,  226,  232,  238,  244,  250,  256,  262,  268,  274,
        280,  286,  292,  298,  304,  310,  316,  322,  328,  334,  340,
        346,  352,  358,  364,  370,  376,  382,  388,  394,  400,  406,
        412,  418,  424,  430,  436,  442,  448,  454,  460,  466,  472,
        478,  484,  490,  496,  502,  508,  514,  520,  526,  532,  538,
        544,  550,  556,  562,  568,  574,  580,  586,  592,  598,  604,
        610,  616,  622,  628,  634,  640,  646,  652,  658,  664,  670,
        676,  682,  688,  694,  700,  706,  712,  718,  724,  730,  736,
        742,  748,  754,  760,  766,  772,  778,  784,  790,  796,  802,
        808,  814,  820,  826,  832,  838,  844,  850,  856,  862,  868,
        874,  880,  886,  892,  898,  904,  910,  9

## Feature 1: Location Plausibility Check 
Give a confidential score about the realibility of location. The scoure is in the range  [0, 4], where for x and y direction range in [0,2] individually.
1. for each unique transmitter ID, create its metrics.

In [48]:
a = data_atk1.loc[(data_atk1['re_ID']==10)]
b = a.loc[(data_atk1['tr_ID']==13)]
print(b.iloc[4]['re_time']-b.iloc[3]['re_time'])
b

1.0


,re_time,re_ID,re_x,re_y,tr_time,tr_ID,tr_x,tr_y,tr_vx,tr_vy,RSSI,Label
382272,21600,10,3584.4,5802.1,21600,13,3587.6,5843.7,-0.029984,30.904,1.180000e-07,0
382291,21601,10,3584.4,5833.6,21601,13,3587.6,5874.6,-0.030080,31.003,4.620000e-07,0
382306,21602,10,3584.3,5865.0,21602,13,3588.5,5905.5,1.315700,30.902,1.820000e-07,0
382323,21603,10,3584.8,5896.4,21603,13,3590.0,5936.5,2.301100,30.904,6.910000e-08,0
382338,21604,10,3586.2,5927.7,21604,13,3592.3,5967.3,2.293700,30.805,5.190000e-08,0
382350,21605,10,3588.4,5958.9,21605,13,3594.6,5998.2,2.302200,30.918,4.460000e-08,0
382360,21606,10,3590.7,5990.1,21606,13,3597.6,6029.0,3.254600,30.820,5.690000e-08,0
382370,21607,10,3593.5,6021.3,21607,13,3600.8,6059.8,3.253900,30.813,4.370000e-08,0


In [90]:
x_95 = [-4.6983, 5.2265]
x_99 = [-7.1795, 7.7077]
y_95 = [-8.1203, 8.0501]
y_99 = [-12.1629, 12.0927]

def Location_plausibility(receiver_of_sender):
    score = 0
    length = receiver_of_sender.shape[0]
    if length == 1:
        return score
    for k in range(length-1):
        time_interval = (receiver_of_sender.iloc[k+1]['re_time'] - receiver_of_sender.iloc[k]['re_time'])
        
        x_pre_95_low = receiver_of_sender.iloc[k]['tr_x'] + time_interval * (receiver_of_sender.iloc[k]['tr_vx'] +  x_95 [0] *time_interval)
        x_pre_95_up = receiver_of_sender.iloc[k]['tr_x'] + time_interval * (receiver_of_sender.iloc[k]['tr_vx'] +  x_95 [1]*time_interval)
        x_pre_99_low = receiver_of_sender.iloc[k]['tr_x'] + time_interval * (receiver_of_sender.iloc[k]['tr_vx'] +  x_99 [0]*time_interval)
        x_pre_99_up = receiver_of_sender.iloc[k]['tr_x'] + time_interval * (receiver_of_sender.iloc[k]['tr_vx'] +  x_99 [1]*time_interval)
        
        y_pre_95_low = receiver_of_sender.iloc[k]['tr_y'] + time_interval * (receiver_of_sender.iloc[k]['tr_vy'] +  y_95 [0]*time_interval)
        y_pre_95_up = receiver_of_sender.iloc[k]['tr_y']  + time_interval * (receiver_of_sender.iloc[k]['tr_vy'] +  y_95 [1]*time_interval)
        y_pre_99_low = receiver_of_sender.iloc[k]['tr_y'] + time_interval * (receiver_of_sender.iloc[k]['tr_vy'] +  y_99 [0]*time_interval)
        y_pre_99_up = receiver_of_sender.iloc[k]['tr_y']  + time_interval * (receiver_of_sender.iloc[k]['tr_vy'] +  y_95 [1]*time_interval)
        
        t_x = 0
        t_y = 0
        if receiver_of_sender.iloc[k+1]['tr_x']<=x_pre_95_low or receiver_of_sender.iloc[k+1]['tr_x'] >= x_pre_95_up:
            t_x = 1
        
        if receiver_of_sender.iloc[k+1]['tr_x']<=x_pre_99_low or receiver_of_sender.iloc[k+1]['tr_x'] >= x_pre_99_up:
            t_x = 2
            
        if receiver_of_sender.iloc[k+1]['tr_y']<=y_pre_95_low or receiver_of_sender.iloc[k+1]['tr_y'] >= y_pre_95_up:
            t_y = 1
        
        if receiver_of_sender.iloc[k+1]['tr_y']<=y_pre_99_low or receiver_of_sender.iloc[k+1]['tr_y'] >= y_pre_99_up:
            t_y = 2  
        
        score = max(score, t_x + t_y)
        
        
    return score

## Feature 2: Movement plausibility check

In [76]:
def Movement_plausibility(receiver_of_sender):
    stationary = 0
    length = receiver_of_sender.shape[0]
    if length == 1:
        return stationary
    x_placement = receiver_of_sender.iloc[length-1]['tr_x'] - receiver_of_sender.iloc[0]['tr_x']
    y_placement = receiver_of_sender.iloc[length-1]['tr_y'] - receiver_of_sender.iloc[0]['tr_y']
    time_duration = (receiver_of_sender.iloc[length-1]['re_time'] - receiver_of_sender.iloc[0]['re_time']) * 0.1
    average_velocity_x = x_placement/time_duration
    average_velocity_y = y_placement/time_duration
    
    
    
    return stationary

SyntaxError: invalid syntax (<ipython-input-76-08a3583d6e16>, line 5)

## Getting the feature

In [93]:
def features(data_atk1):
    count = 0
    
    # feature_vector = np.zeros((data_atk1.shape[0], 6))
    # label = np.zeros(data_atk1.shape[0])
    feature_vector = np.array([])
    
    recevier_ID = np.unique(np.array(data_atk1.iloc[:,1]))
    sender_ID = np.unique(np.array(data_atk1.iloc[:,5]))
    for i in range(len(recevier_ID)):
        sender = data_atk1.loc[(data_atk1['re_ID']==recevier_ID[i])]
        sender_receiver_ID = np.unique(np.array(sender.iloc[:,5]))
        for j in range(len(sender_receiver_ID)):
            receiver_of_sender = data_atk1.loc[(data_atk1['tr_ID']==sender_receiver_ID[j])]
            # Location plausibility check
            feautre_1 = Location_plausibility(receiver_of_sender)
            np.append(feature_vector, feautre_1)
            print(feautre_1)
            print(feature_vector)
            # 
            # feature_vector[count, 1] = 
            # feature_vector[count, 2] = 
            # feature_vector[count, 3] = 
            # feature_vector[count, 4] = 
            # feature_vector[count, 5] = 
            # label[count] = 
            count += 1
    return feature_vector# , label

In [94]:
features(data_atk1)

4
[]
4
[]
4
[]
4
[]
0
[]
4
[]
4
[]
4
[]
4
[]
4
[]
4
[]
4
[]
4
[]
4
[]


KeyboardInterrupt: 

## Feature 2: Movement Plausibility Check 

In [87]:
n = 0
m = np.array([])

In [89]:
np.append(m,n)

array([0.])